# YOLOv11 Veri Seti Tanıtımı

Bu notebook, YOLOv11 formatındaki veri setimizi tanıtmak için hazırlanmıştır. Veri setinin yapısını, örnek görüntüleri, etiketleri ve temel istatistikleri inceleyeceğiz.

## Veri Seti Yapısı
- **Görüntüler**: `dataset/images/` klasöründe `.jpg` formatında.
- **Etiketler**: `dataset/labels/` klasöründe `.txt` formatında, YOLO formatında (`class_id center_x center_y width height`).
- **Sınıflar**: `dataset/classes.txt` dosyasında tanımlı.

## Gereksinimler
- Python kütüphaneleri: `os`, `matplotlib`, `pandas`, `opencv-python`.

Kurulum için:
```bash
pip install opencv-python matplotlib pandas
```

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Veri seti yollarını tanımla
DATASET_PATH = 'Dataset/'
IMAGES_PATH = os.path.join(DATASET_PATH, 'valid/images')
LABELS_PATH = os.path.join(DATASET_PATH, 'valid/labels')
CLASSES_FILE = os.path.join(DATASET_PATH, 'data.yaml')

# Sınıf isimlerini oku
with open(CLASSES_FILE, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

print(f'Sınıflar: {classes}')
print(f'Toplam sınıf sayısı: {len(classes)}')

## Veri Seti İstatistikleri

Görüntü sayısını, etiket sayısını ve sınıf dağılımını hesaplayalım.

In [ ]:
# Görüntü ve etiket dosyalarını listele
image_files = [f for f in os.listdir(IMAGES_PATH) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(LABELS_PATH) if f.endswith('.txt')]

print(f'Toplam görüntü sayısı: {len(image_files)}')
print(f'Toplam etiket dosyası sayısı: {len(label_files)}')

# Sınıf dağılımını hesapla
class_counts = {class_name: 0 for class_name in classes}

for label_file in label_files:
    with open(os.path.join(LABELS_PATH, label_file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            class_counts[classes[class_id]] += 1

# Sınıf dağılımını görselleştir
plt.figure(figsize=(10, 5))
plt.bar(class_counts.keys(), class_counts.values())
plt.title('Sınıf Dağılımı')
plt.xlabel('Sınıf')
plt.ylabel('Örnek Sayısı')
plt.xticks(rotation=45)
plt.show()

## Örnek Görüntü ve Etiket Görselleştirme

Birkaç örnek görüntü ve üzerlerinde etiket kutularını görselleştirelim.

In [ ]:
def draw_bboxes(image_path, label_path, classes):
    # Görüntüyü oku
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    # Etiketleri oku
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())
        class_id = int(class_id)

        # Koordinatları piksele çevir
        x_center *= w
        y_center *= h
        width *= w
        height *= h

        # Sol üst ve sağ alt köşeleri hesapla
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Kutuyu çiz
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        # Sınıf ismini yaz
        cv2.putText(img, classes[class_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return img

# İlk 3 görüntüyü görselleştir
for i, image_file in enumerate(image_files[:5]):
    image_path = os.path.join(IMAGES_PATH, image_file)
    label_path = os.path.join(LABELS_PATH, image_file.replace('.jpg', '.txt'))

    if os.path.exists(label_path):
        img = draw_bboxes(image_path, label_path, classes)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f'Örnek Görüntü {i+1}: {image_file}')
        plt.axis('off')
        plt.show()
    else:
        print(f'Etiket dosyası bulunamadı: {label_path}')